## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key 

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,49.62,49,0,21.85,clear sky
1,1,Mataura,NZ,-46.1927,168.8643,48.96,62,11,5.19,few clouds
2,2,Busselton,AU,-33.6500,115.3333,49.05,90,62,7.72,broken clouds
3,3,Akdepe,TM,42.0551,59.3788,51.89,45,100,7.67,overcast clouds
4,4,Luganville,VU,-15.5333,167.1667,78.48,79,5,5.97,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Luganville,VU,-15.5333,167.1667,78.48,79,5,5.97,clear sky
7,7,Iracoubo,GF,5.4802,-53.2011,81.77,80,56,7.07,broken clouds
9,9,Bambous Virieux,MU,-20.3428,57.7575,75.45,73,40,5.75,scattered clouds
12,12,Mar Del Plata,AR,-38.0023,-57.5575,78.82,65,0,9.22,clear sky
16,16,Isangel,VU,-19.5500,169.2667,75.31,78,3,2.30,clear sky
17,17,Butaritari,KI,3.0707,172.7902,81.41,78,94,7.47,light rain
19,19,Sao Filipe,CV,14.8961,-24.4956,80.82,71,91,11.27,overcast clouds
22,22,Victoria,HK,22.2855,114.1577,75.25,84,67,1.01,light rain
25,25,Indramayu,ID,-6.3264,108.3200,83.71,73,99,4.92,light rain
27,27,Atuona,PF,-9.8000,-139.0333,77.22,68,18,17.36,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()    

City_ID                221
City                   221
Country                218
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()
clean_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp",  "Lat", "Lng", "Current Description" ]].copy()   

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
4,Luganville,VU,78.48,-15.5333,167.1667,clear sky,
7,Iracoubo,GF,81.77,5.4802,-53.2011,broken clouds,
9,Bambous Virieux,MU,75.45,-20.3428,57.7575,scattered clouds,
12,Mar Del Plata,AR,78.82,-38.0023,-57.5575,clear sky,
16,Isangel,VU,75.31,-19.5500,169.2667,clear sky,
17,Butaritari,KI,81.41,3.0707,172.7902,light rain,
19,Sao Filipe,CV,80.82,14.8961,-24.4956,overcast clouds,
22,Victoria,HK,75.25,22.2855,114.1577,light rain,
25,Indramayu,ID,83.71,-6.3264,108.3200,light rain,
27,Atuona,PF,77.22,-9.8000,-139.0333,few clouds,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
    #print(hotel_df) 
    hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
4,Luganville,VU,78.48,-15.5333,167.1667,clear sky,The Espiritu Hotel
7,Iracoubo,GF,81.77,5.4802,-53.2011,broken clouds,Le Gîte d’Iracoubo
9,Bambous Virieux,MU,75.45,-20.3428,57.7575,scattered clouds,Casa Tia Villa
12,Mar Del Plata,AR,78.82,-38.0023,-57.5575,clear sky,Gran Hotel Mar del Plata
16,Isangel,VU,75.31,-19.5500,169.2667,clear sky,Tanna Lodge
...,...,...,...,...,...,...,...
666,Coahuayana,MX,85.17,18.7333,-103.6833,clear sky,El rranchito michuacan de ocampos
670,Cristalina,BR,82.35,-16.7686,-47.6136,broken clouds,HOTEL TOPAZIO
672,Abu Kamal,SY,77.99,34.4506,40.9171,overcast clouds,محطة التلفاز
673,Atar,MR,80.65,20.5169,-13.0499,overcast clouds,Oumou Elghoura


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_df_2= hotel_df.loc[hotel_df["Hotel Name"]!=""] 
clean_df_2

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
4,Luganville,VU,78.48,-15.5333,167.1667,clear sky,The Espiritu Hotel
7,Iracoubo,GF,81.77,5.4802,-53.2011,broken clouds,Le Gîte d’Iracoubo
9,Bambous Virieux,MU,75.45,-20.3428,57.7575,scattered clouds,Casa Tia Villa
12,Mar Del Plata,AR,78.82,-38.0023,-57.5575,clear sky,Gran Hotel Mar del Plata
16,Isangel,VU,75.31,-19.5500,169.2667,clear sky,Tanna Lodge
...,...,...,...,...,...,...,...
666,Coahuayana,MX,85.17,18.7333,-103.6833,clear sky,El rranchito michuacan de ocampos
670,Cristalina,BR,82.35,-16.7686,-47.6136,broken clouds,HOTEL TOPAZIO
672,Abu Kamal,SY,77.99,34.4506,40.9171,overcast clouds,محطة التلفاز
673,Atar,MR,80.65,20.5169,-13.0499,overcast clouds,Oumou Elghoura


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_df_2.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd> <dd> and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df_2.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df_2[["Lat", "Lng"]]

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in clean_df_2.iterrows()]


# 11a. Add a marker layer for each city to the map. 
max_temp = clean_df_2["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))